# Part IV: Validation and Testing

Date: 23/08/2022

Authors: Jordi Bolibar & Facundo Sapienza

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Setup

The goal if this notebook is to explore different validation and testing methodologies that you can use and the respective considerations you have to consider at the moment of asserting the performace of a given model, either for model selection or test report. 

When training and testing a model, we usually split the dataset among three different sets: training, validation and testing set. Sometimes, this split of the data is done authomatically by the method we use, and sometimes instead of a simple split of the dataset into three pieces is done iteratively (eg, in k-fold cross-validation). Independently of how we implement this, we need to understand the limitations and considerations we need to have at the moment we do the split. This considerations can include:
- Sampling procedure
- Spatial and temporal correlations between data points
- Size of data available
- Nature of the datasets: where are they comming from? How independent they are?

How would you desing a validation-testing workflow for the problem of predicting the mass balance?